**ASSIGNMENT 4 - Classification Empirical Study: Text Classification**

**Group Description**

Group Number: 25

Member Name 1: Jake Wang

Member Student Number 1: ***REMOVED***

Member Name 2: Victor Li

Member Student Number 2: ***REMOVED***

**Derived Datasets**

This notebook is a starting point for Assignment 4. In this assignment, you will perform a classification empirical study. This notebook will help you to create derived datasets in Section 2 of the assignment.

In [ ]:
# Let's start by installing spaCy
!pip install spacy

In [ ]:
import spacy
import pandas as pd
import numpy as np

You have been given a list of datasets in the assignment description. Choose one of the datasets and provide the link below and read the dataset using pandas. You should provide a link to your own Github repository even if you are using a reduced version of a dataset from your TA's repository.

TODO: add description of the dataset and your justification of the choices made to obtain the derived datasets

In [ ]:
# url = "https://raw.githubusercontent.com/uOttawa-Collabs/CSI4106-Fall-2023/master/Assignment%204/reduced_drugsComTest_raw_fiveclasses.csv"
# url = "https://raw.githubusercontent.com/uOttawa-Collabs/CSI4106-Fall-2023/master/Assignment%204/reduced_file_AirPassengerReviews.csv"
# url = "https://raw.githubusercontent.com/uOttawa-Collabs/CSI4106-Fall-2023/master/Assignment%204/reduced_file_cnnnews.csv"

In [ ]:
print(url)
data = pd.read_csv(url)

In [ ]:
data.head()

This is where you create the NLP pipeline. load() will download the correct model (English).

In [ ]:
nlp = spacy.load("en_core_web_sm")

Applying the pipeline to every sentences creates a Document where every word is a Token object.

Doc: https://spacy.io/api/doc

Token: https://spacy.io/api/token

In [ ]:
# Apply nlp pipeline to the column that has your sentences.
data['tokenized'] = data['??'].apply(nlp)

In [ ]:
data.head()

A Token object has many attributes such as part-of-speech (pos_), lemma (lemma_), etc. Take a look at the documentation to see all attributes.

The following function is an example on how you can fetch a specific pos tagging from a sentence. We return the lemmatization because we only want the infinitive word.

In [ ]:
# Create empty dataframes that will store your derived datasets
derived_dataset1 = pd.DataFrame(columns=['Class', 'pos'])
derived_dataset2 = pd.DataFrame(columns=['Class', 'pos-np'])

In [ ]:
def get_pos(sentence, wanted_pos):  # wanted_pos refers to the desired pos tagging
    verbs = []
    for token in sentence:
        if token.pos_ in wanted_pos:
            verbs.append(token.lemma_)  # lemma returns a number. lemma_ return a string
    return ' '.join(verbs)  # return value is as a string and not a list for countVectorizer

In [ ]:
# As an example, we use the above function to fetch all the verbs. We store this information in our first derived dataset
derived_dataset1['pos'] = data['tokenized'].apply(lambda sent: get_pos(sent, ['VERB']))

In [ ]:
derived_dataset1.head()

In [ ]:
# Change this line to fetch your desired pos taggings for the second derived dataset
derived_dataset2['pos-np'] = data['tokenized'].apply(??)

In [ ]:
# For Derived Dataset 2, you also need to include Named Entities
# Below is just an example of obtaining such entities on a specific sentence, but you would do NER
#   on the dataset of your choice.
# You can choose the types of entities (dates, organization, people) that you want,
#   and then in your derived dataset, just make sure you include these entities separated by spaces (as shown for verbs)
#   in a previous cell.

sentence = "apple is looking at buying U.K. startup for $1 billion"
doc = nlp(sentence)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Now that you have your derived datasets, you can move to perform your classificaton task.

**Perform Classification Task**